In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os

## Kauppalehti prices and dividends

In [2]:
dividends = pd.read_csv('Osinkohistoria.csv', sep=";", decimal=",", parse_dates=[1], usecols=[0,1,2])
dividends.rename(columns={"Irtoamispäivä": "Päivämäärä", "Osingon määrä":"Osinko"}, inplace=True)
dividends = dividends.groupby(['Osake', 'Päivämäärä'], as_index=False)['Osinko'].sum()
dividends.head()

,Osake,Päivämäärä,Osinko
0,Aallon Group,2020-06-11,0.18
1,Aallon Group,2021-03-19,0.19
2,Admicom,2019-02-04,0.48
3,Admicom,2020-02-03,0.72
4,Admicom,2021-02-22,0.91


In [3]:
price = {}
for filename in os.listdir('kurssihistoria'):
    if filename.endswith(".csv"): 
        stock = os.path.splitext(filename)[0]
        print(stock)
        filepath = os.path.join('kurssihistoria', filename)
        df_stock = pd.read_csv(filepath, sep=';', decimal=',', parse_dates=[0], usecols=[0,3])
        # There are few missing data entries and duplicate dates
        df_stock['Osake'] = stock
        df_stock['Päätöskurssi'].fillna(method='ffill', inplace=True)
        df_stock = df_stock.loc[~df_stock['Päivämäärä'].duplicated(keep='first'),]
        price[stock] = df_stock

Aallon Group
Admicom
Administer
Afarak Group
Affecto
Ahlstrom-Munksjö
Ahlstrom
Ahola Transport A
Ahtium
Aiforia Technologies
Aktia Pankki A
Alexandria Pankkiiriliike
Alma Media
Amer Sports A
Anora Group
Apetit
Aspo
Aspocomp Group
Atria A
Avidly
Basware
BBS-Bioactive Bone Substitutes
Betolar
Bilot
Biohit B
Bioretec
Biotie Therapies
Bittium
Boreo
CapMan
Cargotec
Caverion
Citycon
Componenta
Comptel
Consti
Cramo
Detection Technology
Digia
Digital Workforce
Digitalist Group
DNA
Dovre Group
Duell
EAB Group
EcoUp
Eezy
Efecte
Elecster A
Elisa
Endomines
Enedo
Enento Group
Enersense
eQ
Etteplan
Evli Pankki
Exel Composites
F-Secure
Faron Pharmaceuticals
Fellow Finance
Fifax
Finnair
Fiskars A
FIT Biotech
Fodelia
Fondia
Fortum
Glaston
Gofore
Harvia
Heeros
Herantis Pharma
HKScan A
Hoivatilat
Honkarakenne B
Huhtamäki
Ilkka-Yhtymä 1
Ilkka-Yhtymä 2
Incap
Inderes
Innofactor
Investors House
Kamux
Kemira
Kempower
Keskisuomalainen A
Kesko A
Kesko B
Kesla A
Kojamo
Kone
Konecranes
Kotipizza Group
Kreate Grou

In [4]:
prices = pd.concat(price.values(), axis=0)
prices = prices[~prices['Päätöskurssi'].isnull()]
prices.head()

,Päivämäärä,Päätöskurssi,Osake
0,2021-12-30,11.80,Aallon Group
1,2021-12-29,11.90,Aallon Group
2,2021-12-28,12.05,Aallon Group
3,2021-12-27,11.80,Aallon Group
4,2021-12-23,11.80,Aallon Group


In [5]:
returns = prices.merge(dividends, on=['Osake', 'Päivämäärä'], how='left')
returns.sort_values(['Osake', 'Päivämäärä'], inplace=True)
returns['Osinko'].fillna(0.0, inplace=True)
returns

,Päivämäärä,Päätöskurssi,Osake,Osinko
684,2019-04-09,8.600,Aallon Group,0.0
683,2019-04-10,8.465,Aallon Group,0.0
682,2019-04-11,8.500,Aallon Group,0.0
681,2019-04-12,8.500,Aallon Group,0.0
680,2019-04-15,8.500,Aallon Group,0.0
...,...,...,...,...
658665,2021-12-23,30.000,Ålandsbanken B,0.0
658664,2021-12-27,30.300,Ålandsbanken B,0.0
658663,2021-12-28,30.200,Ålandsbanken B,0.0
658662,2021-12-29,30.500,Ålandsbanken B,0.0


In [6]:
returns['Osakemäärä'] = 1 + returns['Osinko'] / returns['Päätöskurssi']
returns['Osakemäärä'] = returns.groupby('Osake')['Osakemäärä'].cumprod()
returns['Päätöskurssi + Osinko'] = returns['Päätöskurssi'] * returns['Osakemäärä']
returns.head()

,Päivämäärä,Päätöskurssi,Osake,Osinko,Osakemäärä,Päätöskurssi + Osinko
684,2019-04-09,8.600,Aallon Group,0.0,1.0,8.600
683,2019-04-10,8.465,Aallon Group,0.0,1.0,8.465
682,2019-04-11,8.500,Aallon Group,0.0,1.0,8.500
681,2019-04-12,8.500,Aallon Group,0.0,1.0,8.500
680,2019-04-15,8.500,Aallon Group,0.0,1.0,8.500


In [7]:
returns = returns[['Osake', 'Päivämäärä', 'Päätöskurssi', 'Osinko', 'Päätöskurssi + Osinko']]
returns.to_csv("Tuotto.csv", index=False, header=True, date_format='%Y-%m-%d')